In [1]:
import tensorflow as tf
import numpy as np
import pprint

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18
from model.metrics import evaluate

In [2]:
class Config:
    seed = 21
    n_epochs = 20
    batch_size = 128
    embed_dim = 150
    margin = 5
    lr = 0.1

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_train_data(triples_tr, e2idx, p2idx):
    x_s = np.array([e2idx[s] for (s, p, o) in triples_tr], dtype=np.int32)
    x_p = np.array([p2idx[p] for (s, p, o) in triples_tr], dtype=np.int32)
    x_o = np.array([e2idx[o] for (s, p, o) in triples_tr], dtype=np.int32)

    x = {'s': x_s,
         'p': x_p,
         'o': x_o}
    y = np.ones([len(x_s)], dtype=np.float32)
    
    return x, y


def train_input_fn(triples_tr, e2idx, p2idx, random_state, params):
    x, y = build_train_data(triples_tr, e2idx, p2idx)
    s, p, o = x['s'], x['p'], x['o']
    
    s_ = random_state.choice(params['e_vocab_size'], s.shape)
    o_ = random_state.choice(params['e_vocab_size'], o.shape)
    
    x_ = {
        's': np.concatenate([s, s_, s]),
        'p': np.concatenate([p, p, p]),
        'o': np.concatenate([o, o, o_])}
    y_ = np.concatenate([y, np.zeros([2*len(y)], dtype=np.float32)])
    
    return tf.estimator.inputs.numpy_input_fn(x = x_,
                                              y = y_,
                                              batch_size = Config.batch_size,
                                              num_epochs = 1,
                                              shuffle = True)

In [4]:
def forward(features, params):
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    o = tf.nn.embedding_lookup(e_embed, features['o'])
    
    scores = tf.reduce_sum(s * p * o, axis=1)
    
    return scores
    
    
def model_fn(features, labels, mode, params):
    scores = forward(features, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        hinge_loss = tf.nn.relu(Config.margin - scores * (2 * labels - 1))
        loss_op = tf.reduce_sum(hinge_loss)
        train_op = tf.train.AdagradOptimizer(Config.lr).minimize(
            loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = tf.reduce_mean(hinge_loss),
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = scores)

In [5]:
random_state = np.random.RandomState(Config.seed)
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)

model = tf.estimator.Estimator(model_fn,
                               config = tf.estimator.RunConfig(
                                   log_step_count_steps=len(triples_all)//Config.batch_size),
                               params = params)

for _ in range(Config.n_epochs):
    model.train(train_input_fn(triples_tr,
                               e2idx,
                               p2idx,
                               random_state,
                               params))

Files Already Downloaded
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1183, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1096973c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorfl

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt-23212
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 23213 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt.
INFO:tensorflow:loss = 0.55545926, step = 23213
INFO:tensorflow:global_step/sec: 524.898
INFO:tensorflow:loss = 0.112850875, step = 24396 (2.254 sec)
INFO:tensorflow:global_step/sec: 560.874
INFO:tensorflow:loss = 0.70149016, step = 25579 (2.109 sec)
INFO:tensorflow:Saving checkpoints for 26528 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt.
INFO:tensorflow:Loss for final step: 2.780509.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt-49740
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 49741 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt.
INFO:tensorflow:loss = 0.21741642, step = 49741
INFO:tensorflow:global_step/sec: 501.952
INFO:tensorflow:loss = 0.020417895, step = 50924 (2.357 sec)
INFO:tensorflow:global_step/sec: 536.759
INFO:tensorflow:loss = 0.06974356, step = 52107 (2.204 sec)
INFO:tensorflow:Saving checkpoints for 53056 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt.
INFO:tensorflow:Loss for final step: 0.1320281.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p

In [6]:
evaluate(model,
         triples_va,
         triples_te,
         triples_all,
         e2idx,
         p2idx,
         params['e_vocab_size'],
         batch_size = params['e_vocab_size'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt-66320
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
94.9 secs
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpqzzj_8f9/model.ckpt-66320
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
97.6 secs
[valid] Raw Mean Rank: 841.8326
[valid] Raw Hits@1: 42.559999999999995
[valid] Raw Hits@3: 65.32
[valid] Raw Hits@5: 72.59
[valid] Raw Hits@10: 79.38
[valid] Filtered Mean Rank: 831.6222
[valid] Filtered Hits@1: 57.37
[valid] Filtered Hits@3: 77.74
[valid] Filtered Hits@5: 82.31
[valid] Filtered Hits@10: 86.89
INFO:tensorflow:Calling model_fn.
INFO:ten